In [1]:
import pandas as pd
import numpy as np
import scipy
from scipy import stats
import matplotlib.pyplot as plt


In [2]:
directory = "/Users/abittaraev/Desktop/test_task_mail_ru/AB_Test_Results.xlsx"
df = pd.read_excel(directory, header=0)


In [3]:
df.sort_values("USER_ID", inplace = True)
df.reset_index(inplace=True, drop=True)
df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
USER_ID         10000 non-null int64
VARIANT_NAME    10000 non-null object
REVENUE         10000 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 234.5+ KB


,USER_ID,VARIANT_NAME,REVENUE
0,2,control,0.0
1,2,control,0.0
2,2,control,0.0
3,3,variant,0.0
4,3,variant,0.0
5,3,control,0.0
6,4,variant,0.0
7,5,variant,0.0
8,6,variant,0.0
9,9,variant,0.0


Видим, что для некоторых юзеров были применены как экспериментальные условия так и контрольные. Также для некоторых юзеров проводились репликации эксперимента. У нас нет оснований предполагать, что в дизайн эксперимента был заложен план кроссовер, так как нет временных отметок. Поэтому в рамках простого аб-теста безопаснее удалить юзеров, для которых есть обе отметки (и вариант и контроль), а репликации суммировать.

In [4]:
# count treatments for user_id
df_gr=df.groupby('USER_ID')['VARIANT_NAME'].nunique().reset_index()
df_gr.head(10)

,USER_ID,VARIANT_NAME
0,2,1
1,3,2
2,4,1
3,5,1
4,6,1
5,9,1
6,10,2
7,11,1
8,12,1
9,13,1


In [5]:
# find re-used user_ids and remove them
df_to_del = df_gr.loc[df_gr.VARIANT_NAME > 1, ['USER_ID']]
df_mg = pd.merge(df, df_to_del, how='left', on='USER_ID', indicator=True)
df_corr = df_mg.loc[df_mg._merge == 'left_only', ['USER_ID', 'VARIANT_NAME', 'REVENUE']]
df_corr.head(10)

,USER_ID,VARIANT_NAME,REVENUE
0,2,control,0.0
1,2,control,0.0
2,2,control,0.0
6,4,variant,0.0
7,5,variant,0.0
8,6,variant,0.0
9,9,variant,0.0
12,11,control,0.0
13,11,control,0.0
14,12,control,0.0


In [6]:
# sum replicated revenues per user_id
df_sum = df_corr.groupby(['USER_ID', 'VARIANT_NAME'])['REVENUE'].sum().reset_index()
df_sum.head(10)

,USER_ID,VARIANT_NAME,REVENUE
0,2,control,0.0
1,4,variant,0.0
2,5,variant,0.0
3,6,variant,0.0
4,9,variant,0.0
5,11,control,0.0
6,12,control,0.0
7,13,control,0.0
8,15,variant,0.0
9,19,variant,0.0


In [7]:
# summary stats
df_sum.groupby("VARIANT_NAME")["REVENUE"].describe()

VARIANT_NAME       
control       count    2390.000000
              mean        0.196887
              std         4.172201
              min         0.000000
              25%         0.000000
              50%         0.000000
              75%         0.000000
              max       196.010000
variant       count    2393.000000
              mean        0.074935
              std         0.858207
              min         0.000000
              25%         0.000000
              50%         0.000000
              75%         0.000000
              max        23.040000
Name: REVENUE, dtype: float64

Распределения переменной REVENUE (даже если суммировать реплицированные значения по каждому юзеру) определенно ассиметричны. 
Тот факт, что $ \gt 75\% $ наблюдений в обеих группах - это нулевые значения говорит, что наиболее подходящей моделью обоих распределений была бы модель  ZIPR (zero inflated poisson regression). 
В рамках этого эксперимента мы выберем модель попроще. Мы будем считать, что  распределение Пуассона лежит в основе дата-генерационных процессов: $ X_{test} \sim Poisson(\lambda_1) $  и $ X_{control} \sim Poisson(\lambda_2)$ 


In [8]:
#recode REVENUE as binary variable (convert / non-convert) to represent Poisson counts per time unit
df_sum['convert'] = df_sum.REVENUE.apply(lambda x: 1 if x > 0.00  else 0)
df_sum.groupby("VARIANT_NAME")["convert"].describe()

VARIANT_NAME       
control       count    2390.000000
              mean        0.022594
              std         0.148637
              min         0.000000
              25%         0.000000
              50%         0.000000
              75%         0.000000
              max         1.000000
variant       count    2393.000000
              mean        0.017551
              std         0.131341
              min         0.000000
              25%         0.000000
              50%         0.000000
              75%         0.000000
              max         1.000000
Name: convert, dtype: float64

Судя по этим  статистикам, разница в интенсивности конверсии по группма будет незначимой. Следующим шагом, проведем ratratio тест.

Схема тестирования в данном случае будет следующей:
* оценка двух Пуассоновских средних $ \lambda_1 , \lambda_2 $
* rateratio test.
    
Гипотезы rateratio test: 
* $ H_0: \theta = \frac{\lambda_1}{\lambda_2} \leq 1 $
* $ H_a: \theta  = \frac{\lambda_1}{\lambda_2} \gt 1 $
     
В Python нет готового rateratio теста похожего на R, но мы можем  использовать exact binomial test (scipy.stats.binom_test), опираясь на следующее свойство распределения Пуассона: 
* Если $ X_{test} \sim {Poisson(\lambda_1)} $ и  $ X_{control} \sim {Poisson(\lambda_2)} $ независимы, то 
$ X_{test}  \lvert  X_{test} + X_{control} =  k \sim {Binomial(k,p(\theta))} $, где k=количество успешных событий и 
$p(\theta) = \frac{\lambda_1}{\lambda_1 + \lambda_2}$.

Чтобы задать параметры биномиального теста проделаем следующую процедуру:

* Перепишем параметры $\lambda_1,\lambda_2$ как $ r_1 = n\lambda_1, r_2 = m\lambda_2$, где n и m единицы измерения времени.
В нашем случае, для простоты будем считать что за единицу времени может произойти только одно событие convert, поэтому n и m будут равны размеру выборок в группах.
* Поскольку свойства Пуассоновской частоты время-независимы, можем переписать  $\frac{\lambda_1}{\lambda_1 + \lambda_2}$ как $\frac{r_1}{r_1 + r_2}$ => $\frac{n\lambda_1}{n\lambda_1 + m\lambda_2} $
* $\lambda_1= \theta\lambda_2 $, следовательно $\frac{n\lambda_1}{n\lambda_1 + m\lambda_2} 
    = \frac{n\theta\lambda_2}{n\theta\lambda_2 + m\lambda_2} = \frac{n\theta}{n\theta + m}$
*  Таким образом, если $H_0$ верна и $\theta \leq 1$, то тестируем $H_0: \pi = p(\theta)\leq \frac{n}{n+m}$ против $H_a: \pi = p(\theta) \gt \frac{n}{n+m}$, где $\pi$ = обычная пропорция (= вероятность) успехов в биномиальном распределении

In [9]:
#compute sample sizes and pi
dff=df_sum.groupby("VARIANT_NAME")["USER_ID"].count().reset_index()
n = dff.loc[dff["VARIANT_NAME"]=='variant']["USER_ID"].values[0]
m = dff.loc[dff["VARIANT_NAME"]=='control']["USER_ID"].values[0]
pi=n/(n+m)
print("Sample size test: % 2d" %(n), )  
print("Sample size control : % 2d" %(m), )  
print("Proportion of succeses under H_0 : % 5f" %(pi), )  



Sample size test:  2393
Sample size control :  2390
Proportion of succeses under H_0 :  0.500314


In [10]:
#successes in the test group
dff_s=df_sum.groupby("VARIANT_NAME")["convert"].sum().reset_index()
s1= dff_s.loc[dff["VARIANT_NAME"]=='variant']["convert"].values[0]
s2= dff_s.loc[dff["VARIANT_NAME"]=='control']["convert"].values[0]
print("Sample size test: % 2d" %(s1), )  
print("Sample size control: % 2d" %(s2), )  

Sample size test:  42
Sample size control:  54


In [11]:
p_val=scipy.stats.binom_test(s1,s1+s2,pi,alternative="greater")
print("P_value of the test : % 5f" %(p_val), )  
print("Cannot reject H_0")

P_value of the test :  0.908870
Cannot reject H_0


* Разница в конверсиях пользователей статистически не значима,то есть тест не подтверждает  эффективность эксперимента
* Строго говоря эксперимент сам по себе некорректен, так как для чистого эксперимента один юзер может появляться в выборке только один раз и только в одной группе. Строго говоря, эксперимент нужно переделать, принимая эти ограничения во внимание
* Для правильного анализа эксперимента (при наличии большого количества значений переменной revenue=0), нужно использовать более сложные методы, такие как ZIPR